In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('data/final_aggregated_output.csv')

In [14]:
df.dropna(inplace=True)
df.drop(columns=df.columns[-8:], inplace=True)
df.set_index(pd.to_datetime(df['Datetime_linacc']), inplace=True, )
df.drop(columns=[c for c in df.columns if "Datetime" in c], inplace=True)
df.sample(5)

,Time (s),Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Latitude (°),Longitude (°),Height (m),Velocity (m/s),Horizontal Accuracy (m),Vertical Accuracy (m),Acceleration x (m/s^2),Acceleration y (m/s^2),Acceleration z (m/s^2),Gyroscope x (rad/s),Gyroscope y (rad/s),Gyroscope z (rad/s),Activity
Datetime_linacc,,,,,,,,,,,,,,,,,
2024-06-11 14:14:34.499870,304.499869,-1.462701,0.670038,-3.776123,52.331044,4.852079,0.549816,0.00,35.000000,20.991016,2.209461,-7.889433,8.530245,1.361742,1.881591,-2.355655,Sport
2024-06-09 09:56:37.848841,25.848841,4.382508,-4.871615,2.979026,51.936762,4.477387,3.947109,1.29,4.701550,3.423444,-1.657803,-18.414407,-7.960746,2.920400,-1.834929,1.175873,Walking
2024-06-05 18:44:18.668958,49.668959,4.918505,1.911631,0.501836,52.355756,4.927924,0.449124,4.02,7.880651,7.569257,-11.343953,-6.289495,-1.164587,-0.301195,0.096936,1.914357,Cycling
2024-06-04 14:06:56.567799,33.567799,-2.916886,-1.056843,-2.674338,52.333615,4.864405,-1.165651,0.90,5.420729,6.082922,-1.520041,-10.599008,0.446860,1.668821,-0.869496,0.560712,Walking
2024-06-05 18:53:38.215195,267.215184,0.886940,0.013865,-0.409261,52.356098,4.954758,-4.892049,2.62,11.821186,7.090988,-8.374281,-4.264708,-0.388535,0.040639,-0.310915,0.290223,Cycling


In [ ]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


df['dist'] = haversine(df['Latitude (°)'].shift(), df['Longitude (°)'].shift(), df.loc[1:, 'Latitude (°)'], df.loc[1:, 'Longitude (°)'])
df['dist'] = df['dist'].fillna(0)
df['cumdist'] = df['dist'].cumsum() # probably not neeeded
df['displacement'] = haversine(df['Latitude (°)'].iloc[0], df['Longitude (°)'].iloc[0], df['Latitude (°)'].iloc[-1], df['Longitude (°)'].iloc[-1])

In [ ]:
feats = []

for row in df.iloc[:10].itertuples():
    feature_dict = {}
    feature_dict["dt"] = row.index
    print(row)
    feature_dict["accx"] = row["Acceleration x (m/s^2)"]
    feature_dict["accy"] = row["Acceleration y (m/s^2)"]
    feature_dict["accz"] = row["Acceleration z (m/s^2)"]
    feature_dict["gyrox"] = row["Gyroscope x (rad/s)"]
    feature_dict["gyroy"] = row["Gyroscope y (rad/s)"]
    feature_dict["gyroz"] = row["Gyroscope z (rad/s)"]
    feature_dict["lax"] = row["Linear Acceleration x (m/s^2)"]
    feature_dict["lay"] = row["Linear Acceleration y (m/s^2)"]
    feature_dict["laz"] = row["Linear Acceleration z (m/s^2)"]
    feature_dict["velocity"] = row["Velocity (m/s)"]
    feature_dict["height"] = row["Height (m)"]
    #feature_dict["dist"] = row["dist"]
    #feature_dict["disp"] = row["displacement"]
    feats.append(feature_dict)

feats

In [18]:
df.T.to_dict()

C:\Users\yoshm\AppData\Local\Temp\ipykernel_19820\2791767079.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df.T.to_dict()


{Timestamp('2024-06-04 14:06:28.551068'): {'Time (s)': 5.551068034,
  'Linear Acceleration x (m/s^2)': -0.9854925275,
  'Linear Acceleration y (m/s^2)': -0.8658053875,
  'Linear Acceleration z (m/s^2)': 3.814394951,
  'Latitude (°)': 52.33354747,
  'Longitude (°)': 4.86487493,
  'Height (m)': -4.745554336,
  'Velocity (m/s)': 0.9800000191,
  'Horizontal Accuracy (m)': 9.218681335,
  'Vertical Accuracy (m)': 11.65553856,
  'Acceleration x (m/s^2)': 0.147457689,
  'Acceleration y (m/s^2)': 3.445963621,
  'Acceleration z (m/s^2)': 9.239585876,
  'Gyroscope x (rad/s)': -0.02782921307,
  'Gyroscope y (rad/s)': -0.3913399875,
  'Gyroscope z (rad/s)': 0.3730978668,
  'Activity': 'Walking'},
 Timestamp('2024-06-04 14:06:28.591586'): {'Time (s)': 5.591585899,
  'Linear Acceleration x (m/s^2)': -0.8851519823,
  'Linear Acceleration y (m/s^2)': -0.7655408382,
  'Linear Acceleration z (m/s^2)': 3.607431412,
  'Latitude (°)': 52.33354747,
  'Longitude (°)': 4.86487493,
  'Height (m)': -4.745554336,

In [23]:
print("mean acc walking", df[df["Activity"] == "Walking"]['Acceleration x (m/s^2)'].mean())
print("mean acc cycling", df[df["Activity"] == "Cycling"]['Acceleration x (m/s^2)'].mean())
print("mean acc sport", df[df["Activity"] == "Sport"]['Acceleration x (m/s^2)'].mean())

mean acc walking Acceleration x (m/s^2)    0.865319
Acceleration y (m/s^2)   -9.267292
dtype: float64
mean acc cycling -6.045375393442092
mean acc sport -0.06728467307459508
